In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [5]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [6]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'].isin([2013,2014]) == False]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [7]:
# Якутск Орджоникидзе, 56
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [8]:
max_sales=1000
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_reject = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < max_sales)]['item_id'].unique()

In [9]:
train_item_ids = sales_train['item_id'].unique()
train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))

In [10]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [11]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['item_cnt_block'] = training['item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [12]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')


In [13]:
training["shop_cat"] = training["shop_id"].astype(str) + "_" + training["item_category_id"].astype(str)
training["shop_item"] = training["shop_id"].astype(str) + "_" + training["item_id"].astype(str)

In [14]:
unique_shop_cats = training['shop_cat'].unique()
shop_cats = dict(list(zip(unique_shop_cats, range(1,len(unique_shop_cats)))))

def get_shop_cat_int(x):
    if x in shop_cats:
        return shop_cats[x]

training['shop_cat_int'] = training['shop_cat'].apply(lambda x: get_shop_cat_int(x))

In [15]:
unique_shop_items = training['shop_item'].unique()
shop_items = dict(list(zip(unique_shop_items, range(1,len(unique_shop_items)))))

def get_shop_item_int(x):
    if x in shop_items:
        return shop_items[x]

training['shop_item_int'] = training['shop_item'].apply(lambda x: get_shop_item_int(x))

In [16]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
columns = ["item_id", "shop_id", "item_category_id", "month", "shop_cat", "shop_item", "date_block_num"]



y_train = training["item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1
fold 2
fold 3
fold 4
fold 5


In [17]:
def add_block_units_stats(df, cols, name):
    print(name)
    name_units = name + '_units'
    name_mean = name + '_mean'
    name_median = name + '_median'
    name_max = name + '_max'
    name_min = name + '_min'
    name_std = name + '_std'
    
    
    try:
        df.drop(columns=[name_units, name_mean, name_median],inplace=True)
    except:
        pass

    
    block_units = df.groupby(cols,as_index=False)['item_cnt_block'].sum()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_units})
    df = df.merge(block_units, on=cols, how='left')
    df[name_units].fillna(0,inplace=True)
    df[name_units] = pd.to_numeric(df[name_units].astype(int),downcast='unsigned')
    del block_units
    
    block_units_med = df.groupby(cols,as_index=False)['item_cnt_block'].median()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_median})
    df = df.merge(block_units_med, on=cols, how='left')
    df[name_median].fillna(0,inplace=True)
    df[name_median] = pd.to_numeric(df[name_median].astype(int),downcast='unsigned')
    del block_units_med
    
    block_means = df.groupby(cols,as_index=False)['item_cnt_block'].mean()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_mean})
    df = df.merge(block_means, on=cols, how='left')
    df[name_mean].fillna(0,inplace=True)
    df[name_mean] = pd.to_numeric(df[name_mean],downcast='float')
    del block_means
    
    block_max = df.groupby(cols,as_index=False)['item_cnt_block'].max()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_max})
    df = df.merge(block_max, on=cols, how='left')
    df[name_max].fillna(0,inplace=True)
    df[name_max] = pd.to_numeric(df[name_max],downcast='float')
    del block_max
    
    block_min = df.groupby(cols,as_index=False)['item_cnt_block'].min()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_min})
    df = df.merge(block_min, on=cols, how='left')
    df[name_min].fillna(0,inplace=True)
    df[name_min] = pd.to_numeric(df[name_min],downcast='float')
    del block_min
    
    block_std = df.groupby(cols,as_index=False)['item_cnt_block'].std()\
                        .drop_duplicates(cols)\
                        .rename(columns={'item_cnt_block':name_std})
    df = df.merge(block_std, on=cols, how='left')
    df[name_std].fillna(0,inplace=True)
    df[name_std] = pd.to_numeric(df[name_std],downcast='float')
    del block_std
    
    gc.collect()
    return df


training = add_block_units_stats(training, ['item_id','date_block_num'], 'item_block')
training = add_block_units_stats(training, ['shop_id','date_block_num'], 'shop_block')
training = add_block_units_stats(training, ['item_category_id','date_block_num'], 'cat_block')
training = add_block_units_stats(training, ['shop_id', 'item_category_id','date_block_num'], 'shop_cat_block')
training = add_block_units_stats(training, ['shop_id', 'item_id','date_block_num'], 'shop_item_block')

item_block


/opt/conda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:946: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


shop_block
cat_block
shop_cat_block
shop_item_block


In [18]:
def add_rolls(df, cols, name, rolls = [3]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name_tmp})\
            [cols+[roll_name_tmp]]
        
    
        df = df.merge(block_units_rolling_temp, on=cols, how='left')
        #print(df.columns.values)
        del block_units_rolling_temp
        gc.collect()
        

        block_units_rolling = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [roll_name_tmp].shift(1)\
            .rename(columns={roll_name_tmp:roll_name}).reset_index()

        df = df.merge(block_units_rolling, on=cols, how='left')
        df[roll_name].fillna(0,inplace=True)
        df[roll_name] = pd.to_numeric(df[roll_name], downcast='float')
        df.drop(columns=[roll_name_tmp], inplace=True)
        del block_units_rolling
        gc.collect()
    
    return df
    

#training = add_rolls(training, ['item_id','date_block_num'], 'item_block_units')
#training = add_rolls(training, ['item_id','date_block_num'], 'item_block_mean')
#training = add_rolls(training, ['item_id','date_block_num'], 'item_block_median')
#training = add_rolls(training, ['item_id','date_block_num'], 'item_block_min')
#training = add_rolls(training, ['item_id','date_block_num'], 'item_block_max')
#training = add_rolls(training, ['item_id','date_block_num'], 'item_block_std')

#training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_units')
#training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_mean')
#training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_median')
#training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_min')
#training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_max')
#training = add_rolls(training, ['shop_id','date_block_num'], 'shop_block_std')

#training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_units')
#training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_mean')
#training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_median')
#training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_min')
#training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_max')
#training = add_rolls(training, ['item_category_id','date_block_num'], 'cat_block_std')

training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_median')
training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_min')
training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_max')
training = add_rolls(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_std')
#training = add_rolls(training, ['shop_id','item_id','date_block_num'], 'shop_item')

shop_cat_block_units 3
shop_cat_block_mean 3
shop_cat_block_median 3
shop_cat_block_min 3
shop_cat_block_max 3
shop_cat_block_std 3


In [19]:
#training = add_rolls(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

In [20]:
training['block_total'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.sum)

training['item_share_block'] = training['item_block_units'] * 100 / training['block_total']
training['shop_share_block'] = training['shop_block_units'] * 100 / training['block_total']
training['comp2'] = training['item_share_block'] * training['shop_share_block']

In [74]:
def add_lags(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()

        df = df.merge(result, on=cols, how='left')
        df[lag_name].fillna(0,inplace=True)
        if "mean" in name or "std" in name or "share" in name:
            df[lag_name] = pd.to_numeric(df[lag_name], downcast='float')
        else:
            df[lag_name] = pd.to_numeric(df[lag_name].astype(int), downcast='unsigned')
        del result
        gc.collect()
    
    return df
                                         

                                        
#training = add_lags(training, ['item_id','date_block_num'], 'item_block_units')
#training = add_lags(training, ['item_id','date_block_num'], 'item_block_mean')
#training = add_lags(training, ['item_id','date_block_num'], 'item_block_median')                                        
#training = add_lags(training, ['item_id','date_block_num'], 'item_block_min')
#training = add_lags(training, ['item_id','date_block_num'], 'item_block_max')
#training = add_lags(training, ['item_id','date_block_num'], 'item_block_std')

training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_units')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_mean')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_median')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_min')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_max')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_block_std')

#training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_units')
#training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_mean')
#training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_median')
#training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_min')
#training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_max')
#training = add_lags(training, ['item_category_id','date_block_num'], 'cat_block_std')

training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_median')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_min')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_max')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_std')

training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_median')
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_min')
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_max')
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'shop_item_block_std')

shop_block_units 1
shop_block_mean 1
shop_block_median 1
shop_block_min 1
shop_block_max 1
shop_block_std 1
shop_cat_block_units 1
shop_cat_block_mean 1
shop_cat_block_median 1
shop_cat_block_min 1
shop_cat_block_max 1
shop_cat_block_std 1
shop_item_block_units 1
shop_item_block_mean 1
shop_item_block_median 1
shop_item_block_min 1
shop_item_block_max 1
shop_item_block_std 1


In [ ]:
training = add_lags(training, ['item_id','date_block_num'], 'item_share_block')

In [75]:
training = add_lags(training, ['item_id','date_block_num'], 'item_share_block')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_share_block')


item_share_block 1
shop_share_block 1


In [88]:
training['comp2_lag_1'] = training['item_share_block_lag_1'] * training['shop_share_block_lag_1']


In [23]:
total_sum_shops = training.groupby('shop_id')['item_cnt_block'].sum().sum()
training['shop_share'] = training.groupby('shop_id')['item_cnt_block'].transform(np.sum) *100 / total_sum_shops

total_sum_items = training.groupby('item_id')['item_cnt_block'].sum().sum()
training['item_share'] = training.groupby('item_id')['item_cnt_block'].transform(np.sum) *100 / total_sum_items

In [77]:
training['comp1'] = training['shop_share'] * training['item_share']

In [89]:
training[features].sample(10)

,item_id_mean_encoding,shop_cat_mean_encoding,shop_item_block_units_lag_1,shop_cat_pred,shop_cat_block_units_lag_1,cat_pred,comp2_lag_1,shop_cat_block_max_rolling_3,shop_block_mean_lag_1,item_share_block_lag_1,shop_item_block_mean_lag_1
7423,0.136490,0.266867,0,21.225150,6,797.119760,0.007734,4.000000,0.366677,0.007901,0.0
674862,0.596721,0.531457,0,30.170060,28,3979.762874,0.000000,0.000000,0.135643,0.000000,0.0
1143806,0.027211,1.214925,0,50.360479,50,1963.177246,0.000000,14.666667,0.244328,0.000000,0.0
1227944,0.947075,0.490909,1,2.555689,1,169.301796,0.034274,1.000000,0.297799,0.095457,1.0
1350532,1.712329,0.241379,0,10.125000,0,755.916168,0.000000,0.000000,0.000000,0.000000,0.0
772961,0.090652,0.198727,0,99.584431,80,5839.411976,0.000639,4.333333,0.226187,0.005925,0.0
562986,0.256338,0.190275,0,100.871856,105,5006.435928,0.001447,3.000000,0.199284,0.011247,0.0
474855,0.040590,0.133169,0,16.875449,24,1224.159281,0.001633,2.000000,0.308285,0.001875,0.0
1296032,1.440000,0.061706,0,8.208333,10,2215.467066,0.000000,1.500000,0.219557,0.000000,0.0
128708,1.123596,0.566589,3,51.188024,61,3725.177246,0.102846,0.000000,0.368227,0.214685,3.0


In [26]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler


shop_models = {}

for shop_id in all_shop_ids:
    
    shop_data = training[training['shop_id'] == shop_id].groupby(['date_block_num'],as_index=False)['item_cnt_block'].sum()


    regr = linear_model.Ridge()

    X = shop_data['date_block_num'].values.reshape(len(shop_data),1)
    y = shop_data['item_cnt_block'].values.reshape(len(shop_data),1)
    #y = MinMaxScaler().fit_transform(y)

    # Train the model using the training sets
    regr.fit(X, y)
    shop_models[shop_id] = regr

    # Make predictions using the testing set
    #preds = regr.predict(X)

# The coefficients
#print('Coefficients: \n', regr.coef_)
# The mean squared error
#print("Mean squared error: %.2f"
 #     % mean_squared_error(X, preds))
# Explained variance score: 1 is perfect prediction
#print('Variance score: %.2f' % r2_score(X, preds))

# Plot outputs
#plt.scatter(X, y,  color='black')
#plt.plot(X, preds, color='blue', linewidth=3)

#plt.xticks(())
#plt.yticks(())

#plt.show()

print("applying")

def predict(shop_id, dbn):
    return shop_models[shop_id].predict([[dbn]])[0][0]

training['shop_pred'] = training.apply(lambda row: predict(row['shop_id'], row['date_block_num']), axis=1)

applying


In [27]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler


shop_cat_models = {}

for shop_id in all_shop_ids:
    shop_cat_models[shop_id] = {}
    for cat_id in training['item_category_id'].unique():
    
        shop_cat_data = training[(training['shop_id'] == shop_id) & (training['item_category_id'] == cat_id)].groupby(['date_block_num'],as_index=False)['item_cnt_block'].sum()
        if len(shop_cat_data) == 0:
            continue

        regr = linear_model.Ridge()

        X = shop_cat_data['date_block_num'].values.reshape(len(shop_cat_data),1)
        y = shop_cat_data['item_cnt_block'].values.reshape(len(shop_cat_data),1)
        
        #y = MinMaxScaler().fit_transform(y)

        # Train the model using the training sets
        regr.fit(X, y)
        shop_cat_models[shop_id][cat_id] = regr
            

    # Make predictions using the testing set
    #preds = regr.predict(X)


print("applying")

def predict(shop_id, cat_id, dbn):
    return shop_cat_models[shop_id][cat_id].predict([[dbn]])[0][0]

training['shop_cat_pred'] = training.apply(lambda row: predict(row['shop_id'],row['item_category_id'], row['date_block_num']), axis=1)

applying


In [28]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler


cat_models = {}

for cat_id in training['item_category_id'].unique():
    
    cat_data = training[(training['item_category_id'] == cat_id)].groupby(['date_block_num'],as_index=False)['item_cnt_block'].sum()
    if len(cat_data) == 0:
        continue

    regr = linear_model.Ridge()

    X = cat_data['date_block_num'].values.reshape(len(cat_data),1)
    y = cat_data['item_cnt_block'].values.reshape(len(cat_data),1)

    #y = MinMaxScaler().fit_transform(y)

    # Train the model using the training sets
    regr.fit(X, y)
    cat_models[cat_id] = regr


    # Make predictions using the testing set
    #preds = regr.predict(X)


print("applying")

def predict(cat_id, dbn):
    return cat_models[cat_id].predict([[dbn]])[0][0]

training['cat_pred'] = training.apply(lambda row: predict(row['item_category_id'], row['date_block_num']), axis=1)

applying


In [78]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,item_cnt_block,item_category_id,month,shop_cat,shop_item,shop_cat_int,shop_item_int,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding,month_mean_encoding,shop_cat_mean_encoding,shop_item_mean_encoding,date_block_num_mean_encoding,item_block_units,item_block_median,item_block_mean,item_block_max,item_block_min,item_block_std,shop_block_units,shop_block_median,shop_block_mean,shop_block_max,shop_block_min,shop_block_std,cat_block_units,cat_block_median,cat_block_mean,cat_block_max,cat_block_min,cat_block_std,shop_cat_block_units,shop_cat_block_median,shop_cat_block_mean,shop_cat_block_max,shop_cat_block_min,shop_cat_block_std,shop_item_block_units,shop_item_block_median,shop_item_block_mean,shop_item_block_max,shop_item_block_min,shop_item_block_std,shop_cat_block_units_rolling_3,shop_cat_block_mean_rolling_3,shop_cat_block_median_rolling_3,shop_cat_block_min_rolling_3,shop_cat_block_max_rolling_3,shop_cat_block_std_rolling_3,block_total,item_share_block,shop_share_block,comp2,comp2_lag_1,shop_share,item_share,comp1,shop_pred,shop_cat_pred,cat_pred,shop_block_units_lag_1,shop_block_mean_lag_1,shop_block_median_lag_1,shop_block_min_lag_1,shop_block_max_lag_1,shop_block_std_lag_1,shop_cat_block_units_lag_1,shop_cat_block_mean_lag_1,shop_cat_block_median_lag_1,shop_cat_block_min_lag_1,shop_cat_block_max_lag_1,shop_cat_block_std_lag_1,shop_item_block_units_lag_1,shop_item_block_mean_lag_1,shop_item_block_median_lag_1,shop_item_block_min_lag_1,shop_item_block_max_lag_1,shop_item_block_std_lag_1,item_share_block_lag_1,shop_share_block_lag_1
680168,5671,35,27,1,3,4,35_3,35_5671,1857.0,61029.0,1.861972,0.397919,0.843049,0.361467,1.037736,3.222222,0.361467,99,2,2.106383,11,0,0.0,1142,0,0.385160,20,0,0.0,703,0,0.747872,18,0,0.0,22,0,1.100000,5,0,0.0,1,1,1,1,1,0.0,23.333334,1.208772,0.666667,0.0,6.000000,0.0,50362.0,0.196577,0.966284,0.189949,0,2.243961,0.144979,0.325327,1242.680240,21.972455,782.305389,1254,0.431224,0,0,20,0.0,22,1.100000,1,0,5,0.0,1,1.0,1,1,1,0.0,0.095457,1.020438
470956,1610,39,30,0,55,7,39_55,39_1610,762.0,14137.0,0.056980,0.193533,0.259514,0.359144,0.117372,0.000000,0.359144,2,0,0.046512,1,0,0.0,508,0,0.153382,20,0,0.0,6036,0,0.255222,15,0,0.0,56,0,0.101818,3,0,0.0,0,0,0,0,0,0.0,67.000000,0.126265,0.000000,0.0,2.666667,0.0,51454.0,0.003887,0.987290,0.003838,0,1.104483,0.003928,0.004338,654.312575,57.247904,5839.411976,513,0.163428,0,0,20,0.0,71,0.131481,0,0,2,0.0,0,0.0,0,0,0,0.0,0.001975,1.013253
1147994,21592,44,31,0,58,8,44_58,44_21592,926.0,226421.0,0.051515,0.206739,0.052712,0.390454,0.007353,0.000000,0.390454,1,0,0.023810,1,0,0.0,703,0,0.203414,17,0,0.0,182,0,0.042071,3,0,0.0,0,0,0.000000,0,0,0.0,0,0,0,0,0,0.0,1.333333,0.012882,0.000000,0.0,0.666667,0.0,56508.0,0.001770,0.084307,0.000149,0,1.190542,0.003035,0.003614,678.925749,0.995210,234.863473,595,0.179650,0,0,13,0.0,0,0.000000,0,0,0,0.0,0,0.0,0,0,0,0.0,0.003887,1.156373
794647,6399,25,30,0,19,7,25_19,25_6399,1065.0,69122.0,0.447514,1.203570,0.700110,0.359693,1.417241,1.428571,0.359693,10,0,0.232558,2,0,0.0,3449,0,1.041365,20,0,0.0,2873,0,0.596553,14,0,0.0,125,1,1.116071,9,0,0.0,0,0,0,0,0,0.0,143.333328,1.300467,1.000000,0.0,10.666667,0.0,51454.0,0.019435,0.334668,0.006504,0,6.910429,0.031960,0.220855,3951.902994,149.698204,3294.638323,3602,1.147499,0,0,20,0.0,155,1.435185,1,0,15,0.0,2,2.0,2,2,2,0.0,0.027652,0.642320
769508,15402,45,25,0,63,2,45_63,45_15402,2972.0,164758.0,0.323204,0.191633,0.380892,0.455424,0.162371,0.000000,0.455424,16,0,0.340426,3,0,0.0,599,0,0.217739,20,0,0.0,1737,0,0.456265,15,0,0.0,13,0,0.160494,2,0,0.0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,59142.0,0.027054,1.012817,0.027400,0,1.082343,0.028924,0.031306,595.720958,11.008383,1524.885030,668,0.246222,0,0,20,0.0,9,0.118421,0,0,1,0.0,0,0.0,0,0,0,0.0,0.002788,0.017621
830596,3971,2,28,0,55,5,2_55,2_3971,730.0,41105.0,0.183288,0.216556,0.259732,0.374436,0.023093,0.000000,0.374436,5,0,0.113636,2,0,0.0,605,0,0.198948,20

In [30]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'item_cnt_block',
       'item_category_id', 'month', 'shop_cat', 'shop_item',
       'shop_cat_int', 'shop_item_int', 'item_id_mean_encoding',
       'shop_id_mean_encoding', 'item_category_id_mean_encoding',
       'month_mean_encoding', 'shop_cat_mean_encoding',
       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
       'item_block_units', 'item_block_median', 'item_block_mean',
       'item_block_max', 'item_block_min', 'item_block_std',
       'shop_block_units', 'shop_block_median', 'shop_block_mean',
       'shop_block_max', 'shop_block_min', 'shop_block_std',
       'cat_block_units', 'cat_block_median', 'cat_block_mean',
       'cat_block_max', 'cat_block_min', 'cat_block_std',
       'shop_cat_block_units', 'shop_cat_block_median',
       'shop_cat_block_mean', 'shop_cat_block_max', 'shop_cat_block_min',
       'shop_cat_block_std', 'shop_item_block_units',
       'shop_item_block_median', 'shop_item_block_mean',
  

In [31]:
training.fillna(0,inplace=True)
training = training.sample(frac=1).reset_index(drop=True)


In [32]:
gc.collect()

ZEROS_KEEP=0.25

#x_train = training[(training['date_block_num'] < 33) & (training['val_ignore'] == False)]
x_train = training[(training['date_block_num'] < 33)]
y_train = x_train['item_cnt_block']





x_val = training[training['date_block_num'] == 33]
y_val = x_val['item_cnt_block']

pos_val_len = len(y_val[y_val != 0])
print("pos_val_len", pos_val_len)

zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/ZEROS_KEEP)).index
print("zeros_keep_indices_val", len(zeros_keep_indices_val))
non_zeros_val_indices = y_val[y_val != 0].index
print("non_zeros_val_indices", len(non_zeros_val_indices))

val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))

y_val = y_val.loc[val_indices]
x_val = x_val.loc[val_indices]



pos_val_len 29202
zeros_keep_indices_val 116808
non_zeros_val_indices 29202


In [154]:
a = training[['item_cnt_block'] + ['date_block_num','item_id', 'shop_id'] + features].sort_values(by=["date_block_num"]).groupby(["item_id", "shop_id"])#['item_id_mean_encoding']#[features+['item_cnt_block']]

In [169]:
x_train_scaled = MinMaxScaler().fit_transform(a)


KeyboardInterrupt: 

In [160]:
#groups
lstm_data = []
lstm_y = []

for name, group in a:
    #print(group.values)
    steps = []
    ys = []
    #print("group.values",group.values)
    for step in group.values:
        #print("step", len(step))
        steps.append(step[4:])
        #print(step[9])
        ys.append(step[0])
    #remove last
    lstm_data.append(steps[0:8])
    #remove first
    lstm_y.append(ys[1:9])

In [158]:
lstm_data[0]

[array([1.21468927e-01, 1.10429448e-01, 0.00000000e+00, 3.54395210e+01,
        0.00000000e+00, 6.57807066e+03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 array([1.27423823e-01, 1.12791049e-01, 0.00000000e+00, 3.81640719e+01,
        3.60000000e+01, 6.58269940e+03, 2.95583252e-03, 0.00000000e+00,
        2.81607091e-01, 1.95168201e-02, 0.00000000e+00]),
 array([1.21468927e-01, 1.10429448e-01, 0.00000000e+00, 4.08886228e+01,
        3.90000000e+01, 6.58732814e+03, 3.34772724e-03, 3.50000000e+00,
        2.68629581e-01, 2.36718412e-02, 0.00000000e+00]),
 array([1.46067416e-01, 1.17986048e-01, 0.00000000e+00, 4.36131737e+01,
        3.50000000e+01, 6.59195689e+03, 8.61522369e-03, 3.00000000e+00,
        2.03920215e-01, 8.52296967e-03, 0.00000000e+00]),
 array([1.45251397e-01, 1.19607260e-01, 0.00000000e+00, 4.63377246e+01,
        5.00000000e+01, 6.59658563e+03, 8.97359196e-03, 3.00000000e+00,
        1.91905558e-01, 7.94249587e-03, 0.000000

In [161]:
len(features)

11

In [178]:
np.array(lstm_data).reshape(1,232011)

array([[list([array([1.21468927e-01, 1.10429448e-01, 0.00000000e+00, 3.54395210e+01,
       0.00000000e+00, 6.57807066e+03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), array([1.27423823e-01, 1.12791049e-01, 0.00000000e+00, 3.81640719e+01,
       3.60000000e+01, 6.58269940e+03, 2.95583252e-03, 0.00000000e+00,
       2.81607091e-01, 1.95168201e-02, 0.00000000e+00]), array([1.21468927e-01, 1.10429448e-01, 0.00000000e+00, 4.08886228e+01,
       3.90000000e+01, 6.58732814e+03, 3.34772724e-03, 3.50000000e+00,
       2.68629581e-01, 2.36718412e-02, 0.00000000e+00]), array([1.46067416e-01, 1.17986048e-01, 0.00000000e+00, 4.36131737e+01,
       3.50000000e+01, 6.59195689e+03, 8.61522369e-03, 3.00000000e+00,
       2.03920215e-01, 8.52296967e-03, 0.00000000e+00]), array([1.45251397e-01, 1.19607260e-01, 0.00000000e+00, 4.63377246e+01,
       5.00000000e+01, 6.59658563e+03, 8.97359196e-03, 3.00000000e+00,
       1.91905558e-01, 7.94249587e-03, 0.0000000

In [177]:
np.array(lstm_data).shape

(232011,)

In [170]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.callbacks import EarlyStopping

#x_train_scaled = MinMaxScaler().fit_transform(x_train[features])


#x_reshaped = np.reshape(x_train_scaled, (x_train_scaled.shape[0], 10, x_train_scaled.shape[1]))
    
#x_val_scaled_reshaped = np.reshape(x_val_scaled, (x_val_scaled.shape[0], 1, x_val_scaled.shape[1]))


callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=0)
]


my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (10,11)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

history = my_model.fit(lstm_data, lstm_y, batch_size=4096, epochs=100,
                      #validation_data=(x_val_scaled_reshaped,y_val), callbacks=callbacks
                      )



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                19456     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 19,521
Trainable params: 19,521
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 232011 arrays: [array([[1.21468927e-01, 1.10429448e-01, 0.00000000e+00, 3.54395210e+01,
        0.00000000e+00, 6.57807066e+03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]...

In [ ]:
features = [
    
'item_id_mean_encoding',
       'shop_id_mean_encoding', #'item_category_id_mean_encoding',
#       'month_mean_encoding', 'shop_cat_mean_encoding',
#       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
 
#       'item_block_units_rolling_3', 'item_block_mean_rolling_3',
#       'item_block_median_rolling_3', 'item_block_min_rolling_3',
#       'item_block_max_rolling_3', 'item_block_std_rolling_3',
    
#       'shop_block_units_rolling_3', 'shop_block_mean_rolling_3',
#       'shop_block_median_rolling_3', 'shop_block_min_rolling_3',
#       'shop_block_max_rolling_3', 'shop_block_std_rolling_3',
    
#       'cat_block_units_rolling_3', 'cat_block_mean_rolling_3',
#       'cat_block_median_rolling_3', 'cat_block_min_rolling_3',
#       'cat_block_max_rolling_3', 'cat_block_std_rolling_3',
    
       'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3',
       'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3',
       'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3',
    
    
#       'item_block_units_lag_1', 'item_block_mean_lag_1',
#       'item_block_median_lag_1', 'item_block_min_lag_1',
#       'item_block_max_lag_1', 'item_block_std_lag_1',
    
#       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
#       'shop_block_median_lag_1', 'shop_block_min_lag_1',
       'shop_block_max_lag_1', 'shop_block_std_lag_1',
    
#      'cat_block_units_lag_1', 'cat_block_mean_lag_1',
#       'cat_block_median_lag_1', 'cat_block_min_lag_1',
#       'cat_block_max_lag_1', 'cat_block_std_lag_1',
    
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
#       'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1',
#       'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1',
    
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
#       'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1',
#       'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1',
    
       'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1',
    
    'shop_pred', 'shop_cat_pred', 'cat_pred'

]


In [60]:
features = ['item_id_mean_encoding',
 'shop_cat_mean_encoding',
 'shop_item_block_units_lag_1',
 'shop_cat_pred',
 'shop_cat_block_units_lag_1',
 'cat_pred',
 'comp2_lag_1',
 'shop_cat_block_max_rolling_3',
 'shop_block_mean_lag_1',
 'item_share_block_lag_1',
 'shop_item_block_mean_lag_1']

In [40]:
from sklearn.preprocessing import MinMaxScaler

x_train_scaled = MinMaxScaler().fit_transform(x_train[features])
x_val_scaled = MinMaxScaler().fit_transform(x_val[features])

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [63]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor


lr_model =  BayesianRidge()
lr_model.fit(x_train_scaled, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_scaled)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

rmse:  1.0587979027152195


In [ ]:
lr_model.coef_

In [45]:
#test            = pd.read_csv('test.csv.gz')
test = test.set_index('item_id').join(items.set_index('item_id'))
test.reset_index(inplace=True)

In [46]:
def predict(shop_id):
    return shop_models[shop_id].predict([[34]])[0][0]

#test['shop_pred'] = test.apply(lambda row: predict(row['shop_id']), axis=1)

def predict(shop_id, cat_id):
    if shop_id in shop_cat_models and cat_id in shop_cat_models[shop_id]:
        return shop_cat_models[shop_id][cat_id].predict([[34]])[0][0]

test['shop_cat_pred'] = test.apply(lambda row: predict(row['shop_id'],row['item_category_id']), axis=1)

def predict(cat_id):
    if cat_id in cat_models:
        return cat_models[cat_id].predict([[34]])[0][0]

test['cat_pred'] = test.apply(lambda row: predict(row['item_category_id']), axis=1)

In [47]:
item_features = [ 
    'item_id_mean_encoding'
                ]

merge_col = ['item_id']
cols=item_features+merge_col

test = test.merge(training.drop_duplicates('item_id')[cols], on=merge_col, how='left')

In [ ]:
shop_features = [
        'shop_id_mean_encoding'
]

merge_col = ['shop_id']
cols=shop_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [ ]:
cat_features = [
        'item_category_id_mean_encoding'#,'cat_me_real'
]

merge_col = ['item_category_id']
cols=cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [ ]:
shop_item_features = [
        'shop_item_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_item_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [53]:
shop_cat_features = [
        'shop_cat_mean_encoding'
]

merge_col = ['shop_id', 'item_id']
cols=shop_cat_features+merge_col


test = test.merge(training.drop_duplicates(merge_col)[cols], on=merge_col, how='left')

In [48]:
def add_rolls_test(df, cols, name, rolls = [3]):
    for roll in rolls:
        print(name, roll)
        roll_name = name+"_rolling_" + str(roll)
        roll_name_tmp = roll_name + "_tmp"
        
        try:
            df.drop(columns=[roll_name],inplace=True)
        except:
            pass       

    
        block_units_rolling_temp = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].rolling(roll,min_periods=2).mean().reset_index()\
            .rename(columns={name:roll_name})\
            [cols+[roll_name]]
        
        print([cols[0:len(cols)-1]+[roll_name]])
        thirty_three = block_units_rolling_temp[block_units_rolling_temp['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1]+[roll_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')
    

        del block_units_rolling_temp
        gc.collect()
        

    
    return df
    

#test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_mean')
#test = add_rolls_test(test, ['item_id','date_block_num'], 'item_block_units')
#test = add_rolls_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')

#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_median')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_min')
test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_max')
#test = add_rolls_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_std')

shop_cat_block_max 3
[['shop_id', 'item_category_id', 'shop_cat_block_max_rolling_3']]


In [ ]:
test = add_rolls_test(test, ['item_category_id','date_block_num'], 'cat_block_mean')

In [ ]:
test = add_rolls_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

In [49]:
def add_lags_test(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = training\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()
        
        thirty_three = result[result['date_block_num'] == 33].drop_duplicates(cols)\
                [cols[0:len(cols)-1] + [lag_name]]
        df = df.merge(thirty_three, on=cols[0:len(cols)-1], how='left')

        gc.collect()
    
    return df
                                         

                                        
#test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_mean')
#test = add_lags_test(test, ['item_id','date_block_num'], 'item_block_units')
test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_mean')
#test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_mean')

#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_max')
#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_block_std')

test = add_lags_test(test, ['shop_id','item_category_id','date_block_num'], 'shop_cat_block_units')

test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')

test = add_lags_test(test, ['item_id','date_block_num'], 'item_share_block')
#test = add_lags_test(test, ['shop_id','date_block_num'], 'shop_share_block')
#test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'comp2')

shop_block_mean 1
shop_cat_block_units 1
shop_item_block_units 1
shop_item_block_mean 1
item_share_block 1


In [ ]:
test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_mean')
#test = add_lags_test(test, ['shop_id','item_id','date_block_num'], 'shop_item_block_units')

In [50]:
#training['comp2'] = training['item_share_block'] * training['shop_share_block']


test = add_lags_test(test, ['item_id','shop_id','date_block_num'], 'comp2')

comp2 1


In [54]:
test.fillna(0, inplace=True)

In [55]:
from sklearn.preprocessing import MinMaxScaler

test_scaled = MinMaxScaler().fit_transform(test[features])


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [64]:
preds = lr_model.predict(test_scaled)
preds.clip(0,20,out=preds)


array([ 0.        ,  0.        ,  0.15157035, ...,  0.26322811,
        0.28951454,  0.21382811])

In [92]:
preds = lg_model.predict(test[features])
preds.clip(0,20,out=preds)


array([ 0.11804522,  0.07073731,  0.18383498, ...,  0.04995914,
        0.28776269,  0.09947505])

In [96]:

print(np.mean(preds))
print(np.max(preds))

0.291327336924
17.0728554221


In [ ]:
preds = np.mean(np.array([lg_preds, lr_preds]),axis=0)

In [97]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [95]:
lr_preds = pd.read_csv('lr110.csv')['item_cnt_month']
lg_preds = pd.read_csv('lg110.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lg_preds * 0.50) + (lr_preds * 0.50)

In [ ]:
training.columns.values

In [30]:
all_features = ['item_id_mean_encoding',
       'shop_id_mean_encoding', 'item_category_id_mean_encoding',
       'month_mean_encoding', 'shop_cat_mean_encoding',
       'shop_item_mean_encoding', 'date_block_num_mean_encoding',
       'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3',
       'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3',
       'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3',
       'shop_block_units_lag_1', 'shop_block_mean_lag_1',
       'shop_block_median_lag_1', 'shop_block_min_lag_1',
       'shop_block_max_lag_1', 'shop_block_std_lag_1',
       'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1',
       'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1',
       'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1',
       'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1',
       'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1',
       'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1',
       'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1',
       'shop_share', 'item_share', 'shop_pred', 'shop_cat_pred',
       'cat_pred']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Create a random forest classifier
clf = RandomForestRegressor(n_estimators=10, random_state=0, n_jobs=8)

# Train the classifier
clf.fit(x_train[all_features], y_train)

# Print the name and gini importance of each feature


In [ ]:
f_ = sorted(list(zip(all_features, clf.feature_importances_)), key=lambda x: x[1])[::-1][0:]

features = [f[0] for f in f_]

from sklearn.preprocessing import MinMaxScaler

x_train_scaled = MinMaxScaler().fit_transform(x_train[features])
x_val_scaled = MinMaxScaler().fit_transform(x_val[features])


lr_model =  Ridget(alpha=0)
lr_model.fit(x_train_scaled, y_train)

from sklearn.metrics import mean_squared_error
from math import sqrt

lr_val_preds = lr_model.predict(x_val_scaled)
lr_val_preds.clip(0,20,out=lr_val_preds)
rms = sqrt(mean_squared_error(y_val, lr_val_preds))
print("rmse: ", rms)

In [ ]:
sorted(list(zip(all_features, clf.feature_importances_)), key=lambda x: x[1])[::-1]

In [ ]:
all_features

In [38]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor
import math

all_f = all_features.copy()
keep = []

print(all_f)

best = 10000
while len(all_f) > 0:
    print("keep", keep)
    len_keep = len(keep)
    #print(len(all_f))
    res = {}
    for idx,feat in enumerate(all_f):

        potential = keep + [feat]

        x_train_scaled = MinMaxScaler().fit_transform(x_train[potential])
        x_val_scaled = MinMaxScaler().fit_transform(x_val[potential])


        lr_model =  Ridge(alpha=0)
        lr_model.fit(x_train_scaled, y_train)


        lr_val_preds = lr_model.predict(x_val_scaled)
        lr_val_preds.clip(0,20,out=lr_val_preds)
        rms = math.sqrt(mean_squared_error(y_val, lr_val_preds))

        res[feat] = rms
        
    sorted_by_value = sorted(res.items(), key=lambda kv: kv[1])
    best_score = sorted_by_value[0][1]
    best_feature = sorted_by_value[0][0]
    print("best_score", best_score)
    print("best_feature", best_feature)
    if best_score < best:
        best = best_score
        keep.append(best_feature)
        all_f.remove(best_feature)
    else:
        break
    
keep

['item_id_mean_encoding', 'shop_id_mean_encoding', 'item_category_id_mean_encoding', 'month_mean_encoding', 'shop_cat_mean_encoding', 'shop_item_mean_encoding', 'date_block_num_mean_encoding', 'shop_cat_block_units_rolling_3', 'shop_cat_block_mean_rolling_3', 'shop_cat_block_median_rolling_3', 'shop_cat_block_min_rolling_3', 'shop_cat_block_max_rolling_3', 'shop_cat_block_std_rolling_3', 'shop_block_units_lag_1', 'shop_block_mean_lag_1', 'shop_block_median_lag_1', 'shop_block_min_lag_1', 'shop_block_max_lag_1', 'shop_block_std_lag_1', 'shop_cat_block_units_lag_1', 'shop_cat_block_mean_lag_1', 'shop_cat_block_median_lag_1', 'shop_cat_block_min_lag_1', 'shop_cat_block_max_lag_1', 'shop_cat_block_std_lag_1', 'shop_item_block_units_lag_1', 'shop_item_block_mean_lag_1', 'shop_item_block_median_lag_1', 'shop_item_block_min_lag_1', 'shop_item_block_max_lag_1', 'shop_item_block_std_lag_1', 'item_share_block_lag_1', 'shop_share_block_lag_1', 'comp2_lag_1', 'shop_share', 'item_share', 'shop_pred

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return

best_score 1.2213909415288133
best_feature item_id_mean_encoding
keep ['item_id_mean_encoding']


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype float32, float64 were

...

best_score 1.0587356513519661
best_feature shop_item_block_mean_lag_1
keep ['item_id_mean_encoding', 'shop_cat_mean_encoding', 'shop_item_block_units_lag_1', 'shop_cat_pred', 'shop_cat_block_units_lag_1', 'cat_pred', 'comp2_lag_1', 'shop_cat_block_max_rolling_3', 'shop_block_mean_lag_1', 'item_share_block_lag_1', 'shop_item_block_mean_lag_1']


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataCo

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, uint16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataCo

best_score 1.0587553332327528
best_feature shop_share_block_lag_1


['item_id_mean_encoding',
 'shop_cat_mean_encoding',
 'shop_item_block_units_lag_1',
 'shop_cat_pred',
 'shop_cat_block_units_lag_1',
 'cat_pred',
 'comp2_lag_1',
 'shop_cat_block_max_rolling_3',
 'shop_block_mean_lag_1',
 'item_share_block_lag_1',
 'shop_item_block_mean_lag_1']

In [ ]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_by_value = sorted(x.items(), key=lambda kv: kv[1])[::-1]
sorted_by_value

In [91]:
gc.collect()
lgtrain = lgbm.Dataset(x_train[features], label=y_train)
lgval = lgbm.Dataset(x_val[features], label=y_val)


params = {
        "num_threads": 16,
        #"device": "gpu",
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        "max_bin": 30,#default 255
        #"num_leaves": 70, #default 31
        "bagging_fraction": 0.5,
        "bagging_freq": 1,
        "min_data_in_leaf": 2500,
        "feature_fraction": 0.7,
        #"lambda_l2": 3,
        #"max_depth": 7,
        #"min_gain_to_split": 10,
        "learning_rate" : 0.1,
        #"histogram_pool_size": 1000,
        #"categorical_column": [0,1,2,3,4]
}

evals_result = {}
lg_model = lgbm.train(params, lgtrain, 20000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=10, 
                      verbose_eval=10, 
                      evals_result=evals_result)

scores = {}
for i,score in enumerate(lg_model.feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's rmse: 1.16893
[20]	valid_0's rmse: 1.12071
[30]	valid_0's rmse: 1.10703
[40]	valid_0's rmse: 1.10052
[50]	valid_0's rmse: 1.09479
[60]	valid_0's rmse: 1.08897
[70]	valid_0's rmse: 1.08548
[80]	valid_0's rmse: 1.08165
[90]	valid_0's rmse: 1.07856
[100]	valid_0's rmse: 1.07572
[110]	valid_0's rmse: 1.07395
[120]	valid_0's rmse: 1.0719
[130]	valid_0's rmse: 1.07138
Early stopping, best iteration is:
[123]	valid_0's rmse: 1.07102


[('shop_cat_mean_encoding', 579),
 ('shop_cat_pred', 538),
 ('item_id_mean_encoding', 532),
 ('cat_pred', 525),
 ('shop_block_mean_lag_1', 398),
 ('shop_item_block_units_lag_1', 360),
 ('shop_cat_block_max_rolling_3', 333),
 ('shop_cat_block_units_lag_1', 307),
 ('shop_item_block_mean_lag_1', 118),
 ('item_share_block_lag_1', 0),
 ('comp2_lag_1', 0)]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor()

neigh.fit(x_train_scaled, y_train) 